# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [ ]:
import os

from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig
from azureml.core.dataset import Dataset

from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

from azureml.core.model import InferenceConfig
from azureml.core.environment import Environment
from azureml.core.webservice import AciWebservice

import requests
import json

# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

In [ ]:
ws = Workspace.from_config()

# Choosing a name for experiment
experiment_name = 'AutoML-Experiment'

experiment=Experiment(ws, experiment_name)
#project_folder = './pipeline-project'


# Compute Cluster

In [ ]:
cpu_cluster_name = "cpu-cluster"

# Verifying that cluster does not exist already
try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',
                                                          max_nodes=4)
    cpu_cluster = ComputeTarget.create(ws, cpu_cluster_name, compute_config)

cpu_cluster.wait_for_completion(show_output=True)

# Dataset


TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


The dataset "Heart Failure Prediction" contains 12 clinical features of 299 patients with heart failure and a target variable "DEATH EVENT" indicating if the patient deceased during the follow-up period (boolean). Machine Learning can help detect and manage high-risk patients at an earlier stage. 

To solve this binary classification problem, I will use the 12 features to predict possible death events with the help of an ML algorithm.  


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [ ]:
found = False
key = "Heart Failure Prediction"
description_text = "Heart Failure Prediction DataSet for Udacity Project 3"

if key in ws.datasets.keys(): 
        found = True
        dataset = ws.datasets[key] 

if not found:
        # Creating Dataset and register it into Workspace
        example_data = 'https://github.com/EugeniaSilantjeva/nd00333-capstone/blob/master/heart_failure_clinical_records_dataset.csv'
        dataset = Dataset.Tabular.from_delimited_files(example_data)        
        # Registering Dataset in Workspace
        dataset = dataset.register(workspace=ws,
                                   name=key,
                                   description=description_text)

In [ ]:
dataset.take(5).to_pandas_dataframe()

In [ ]:
df = dataset.to_pandas_dataframe()
df.describe()

In [ ]:
df[df.isnull().any(axis=1)]

In [ ]:
df["DEATH_EVENT"].value_counts()

In [ ]:
# TODO: Put your automl settings here
automl_settings = {
    "experiment_timeout_minutes": 20,
    "max_concurrent_iterations": 4,
    "primary_metric" : 'AUC_weighted'
}

# TODO: Put your automl config here
automl_config = AutoMLConfig(compute_target=cpu_cluster,
                             task = "classification",
                             training_data=dataset,
                             label_column_name="DEATH_EVENT",   
                             enable_early_stopping= True,
                             featurization= 'auto',
                             debug_log = "automl_errors.log",
                             **automl_settings
                            )


In [ ]:
# TODO: Submit your experiment
automl_run = experiment.submit(automl_config)

## Run Details


TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [ ]:
RunDetails(automl_run).show()
automl_run.wait_for_completion(show_output = True)
assert(automl_run.get_status() == "Completed")

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [ ]:
# Retrieving and saving the best automl model
automl_best_run, fitted_automl_model = automl_run.get_output() 

run_dict = automl_best_run.get_details()

print("Best Run ID: ", run_dict["runId"])
print("---"*10)
#print("Hyperparameters of the Best Model: ", fitted_automl_model.steps[1][1])
#print("---"*10)
#print("Hyperparameters: ", model.steps[-1].get_params())
#print("Hyperparameters: ", model.steps[1][1].get_params())
print("---"*10)
print("Fitted model: ", fitted_automl_model.steps[-1])
print("---"*10)
print("AutoML best run AUC_weighted: ", automl_best_run.get_metrics(name="AUC_weighted"))
print("---"*10)
print("Metrics: ", automl_best_run.get_metrics())
print("---"*10)
print("AutoML Run Summary: ", automl_run.summary())

In [ ]:
#TODO: Save the best model
os.makedirs('outputs', exist_ok=True)

model = automl_best_run.register_model(model_name = 'automl_best_model.pkl', model_path = './outputs/')

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [ ]:
automl_best_run.get_file_names()

In [ ]:
# Reference: https://docs.microsoft.com/en-us/azure/machine-learning/how-to-deploy-existing-model
# https://github.com/Azure/MachineLearningNotebooks/blob/master/how-to-use-azureml/deployment/deploy-to-cloud/model-register-and-deploy.ipynb

# Downloading the scoring file and conda environment to the same directory where I have saved the trained model
scoring_file = ""
conda_env = ""

best_run.download_file('outputs/' + scoring_file , 'score.py')
best_run.download_file('outputs/' + conda_env, 'env.yml')

# Creating the environment
env=Environment.from_conda_specification(name='env',file_path = "env.yml")

# Creating an inference config
inference_config = InferenceConfig(entry_script="score.py",
                                   environment=env)

# Deploying the model as a web service to an Azure Container Instance (ACI)
service_name = 'Heart-Failure-Prediction'
aci_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb=1)

deployed_model_webservice = Model.deploy(workspace=ws,
                       name=service_name,
                       models=[model],
                       inference_config=inference_config,
                       deployment_config=aci_config,
                       overwrite=True)

deployed_model_webservice.wait_for_deployment(show_output=True)



TODO: In the cell below, send a request to the web service you deployed to test it.

In [ ]:
# Reference: https://github.com/Azure/MachineLearningNotebooks/blob/master/how-to-use-azureml/deployment/deploy-to-cloud/model-register-and-deploy.ipynb 
# and endpoint.py file used in Project 2


scoring_uri = 'http://4602d749-61d8-4cee-87ae-ce283ac3fc36.southcentralus.azurecontainer.io/score'
# If the service is authenticated, set the key or token
# key = 'srl50WQIAoblswO8iYyRAovQiA74twp6'


df.drop(columns=["DEATH_EVENT"])

input_data = json.dumps({
    'data': df[0:2].to_dict()
})

with open("data.json", "w") as _f:
    _f.write(input_data)

# Set the content type
headers = {'Content-Type': 'application/json'}
# If authentication is enabled, set the authorization header
# headers['Authorization'] = f'Bearer {key}'

# Make the request and display the response
resp = requests.post(scoring_uri, input_data, headers=headers)
print(resp.json())    
    

TODO: In the cell below, print the logs of the web service and delete the service

In [ ]:
# Retrieve logs for this Webservice
deployed_model_webservice.get_logs()

# Delete service
deployed_model_webservice.delete()